In [ ]:
import sys
sys.path.insert(0, r'G:\projects\GitHub\HE\HEMnet')

import os
import pandas as pd
import numpy as np
import SimpleITK as sitk
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

from normaliser import IterativeNormaliser
from utils import *

In [ ]:
os.makedirs(OUTPUT_PATH, exist_ok=True)
to_handle = list(HE_KI67_SLIDES_CUT_INFO_PAIR.keys())

HE_slides = [slide+HE_NAME for slide in to_handle]
KI67_slides = [slide+KI67_NAME for slide in to_handle]
Paired_slides = list(zip(KI67_slides, HE_slides))
for i, pair in enumerate(Paired_slides):
    ki67, he = pair
    verbose_print('{0}. {1}|{2}'.format(i + 1, ki67, he))

In [ ]:
NORMALISER_METHOD = 'vahadane'
STANDARDISE_LUMINOSITY = True
normaliser = IterativeNormaliser(NORMALISER_METHOD, STANDARDISE_LUMINOSITY)

template_img = read_kfb_region(TEMPLATE_SLIDE_NAME, True)
normaliser.fit_target(template_img)
thumbnail(template_img)

In [ ]:
performance_df = pd.DataFrame((Paired_slides), columns=["KI67_Slide_Name", "H&E_Slide_Name"])

for idx,slide_name in enumerate(to_handle):
    print('-' * (18 + len(slide_name)))
    print('Processing Slide: {0}'.format(slide_name))
    he_name = slide_name + HE_NAME
    ki67_name = slide_name + KI67_NAME

    he = read_kfb_region(slide_name, True)
    normaliser.fit_source(he)
    he_norm = normaliser.transform_tile(he)
    verbose_save_img(he_norm.convert('RGB'), os.path.join(OUTPUT_PATH, slide_name+'_normalised.jpeg'), 'JPEG')
    
    ki67 = read_kfb_region(slide_name, False)
    verbose_save_img(ki67.convert('RGB'), os.path.join(OUTPUT_PATH, slide_name+'_KI67.jpeg'), 'JPEG')

    INTERPOLATOR = sitk.sitkLanczosWindowedSinc

    # Convert to grayscale
    ki67_gray = ki67.convert('L')
    he_gray = he_norm.convert('L')
    # Convert to ITK format
    ki67_itk = get_itk_from_pil(ki67_gray)
    he_itk = get_itk_from_pil(he_gray)
    # Set fixed and moving images
    fixed_img = he_itk
    moving_img = ki67_itk

    # Check initial registration ################################################
    # Centre the two images, then compare their alignment
    initial_transform = sitk.CenteredTransformInitializer(fixed_img, moving_img, sitk.Euler2DTransform(),
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    moving_rgb = sitk_transform_rgb(ki67, he_norm, initial_transform)

    # Compute the mutual information between the two images before registration
    moving_resampled_initial = sitk.Resample(moving_img, fixed_img, initial_transform,
                                             INTERPOLATOR, 0.0, moving_img.GetPixelID())
    initial_mutual_info = calculate_mutual_info(np.array(he_gray),
                                                np.array(get_pil_from_itk(moving_resampled_initial)))
    verbose_print('Initial mutual information metric: {0}'.format(initial_mutual_info))
    performance_df.loc[idx, "Initial_Mutual_Info"] = initial_mutual_info
    

    # --- Affine Registration --- #  ################################################
    initial_transform = sitk.CenteredTransformInitializer(fixed_img, moving_img, sitk.Euler2DTransform(),
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    affine_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    affine_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    affine_method.SetMetricSamplingStrategy(affine_method.RANDOM)
    affine_method.SetMetricSamplingPercentage(0.15)

    affine_method.SetInterpolator(INTERPOLATOR)

    # Optimizer settings.
    affine_method.SetOptimizerAsGradientDescent(learningRate=1, numberOfIterations=100,
                                                convergenceMinimumValue=1e-6, convergenceWindowSize=20)
    affine_method.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework.
    affine_method.SetShrinkFactorsPerLevel(shrinkFactors=[8, 4])
    affine_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[4, 2])
    affine_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Don't optimize in-place, we would possibly like to run this cell multiple times.
    affine_method.SetInitialTransform(initial_transform, inPlace=False)

    # Connect all of the observers so that we can perform plotting during registration.
    affine_method.AddCommand(sitk.sitkStartEvent, start_plot)
    affine_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, update_multires_iterations)
    affine_method.AddCommand(sitk.sitkIterationEvent, lambda: update_plot(affine_method))

    affine_transform = affine_method.Execute(sitk.Cast(fixed_img, sitk.sitkFloat32),
                                             sitk.Cast(moving_img, sitk.sitkFloat32))

    affine_fig = plot_metric('Plot of mutual information cost in affine registration')
    plt.show()
    verbose_save_fig(affine_fig, os.path.join(OUTPUT_PATH, slide_name + '_affine_metric_plot.jpeg'))
    end_plot()

    verbose_print(
        'Affine Optimizer\'s stopping condition, {0}'.format(
            affine_method.GetOptimizerStopConditionDescription()))
    

    # Compute the mutual information between the two images after affine registration
    moving_resampled_affine = sitk.Resample(moving_img, fixed_img, affine_transform,
                                            INTERPOLATOR, 0.0, moving_img.GetPixelID())
    affine_mutual_info = calculate_mutual_info(np.array(he_gray),
                                               np.array(get_pil_from_itk(moving_resampled_affine)))
    verbose_print('Affine mutual information metric: {0}'.format(affine_mutual_info))
    performance_df.loc[idx, "Affine_Mutual_Info"] = affine_mutual_info

    
    # --- B-spline registration --- #  ################################################

    bspline_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    bspline_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    bspline_method.SetMetricSamplingStrategy(bspline_method.RANDOM)
    bspline_method.SetMetricSamplingPercentage(0.15)

    bspline_method.SetInterpolator(INTERPOLATOR)

    # Optimizer settings.
    bspline_method.SetOptimizerAsGradientDescent(learningRate=1, numberOfIterations=200,
                                                 convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    bspline_method.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework.
    bspline_method.SetShrinkFactorsPerLevel(shrinkFactors=[2, 1])
    bspline_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[1, 0])
    bspline_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Don't optimize in-place, we would possibly like to run this cell multiple times.
    transformDomainMeshSize = [8] * moving_resampled_affine.GetDimension()
    initial_transform = sitk.BSplineTransformInitializer(fixed_img, transformDomainMeshSize)
    bspline_method.SetInitialTransform(initial_transform, inPlace=False)

    # Connect all of the observers so that we can perform plotting during registration.
    bspline_method.AddCommand(sitk.sitkStartEvent, start_plot)
    bspline_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, update_multires_iterations)
    bspline_method.AddCommand(sitk.sitkIterationEvent, lambda: update_plot(bspline_method))

    bspline_transform = bspline_method.Execute(sitk.Cast(fixed_img, sitk.sitkFloat32),
                                               sitk.Cast(moving_resampled_affine, sitk.sitkFloat32))

    bspline_fig = plot_metric('Plot of mutual information cost in B-spline registration')
    plt.show()
    verbose_save_fig(bspline_fig, os.path.join(OUTPUT_PATH, slide_name + '_bspline_metric_plot.jpeg'))
    end_plot()

    verbose_print('B-spline Optimizer\'s stopping condition, {0}'.format(
        bspline_method.GetOptimizerStopConditionDescription()))

    # Compute the mutual information between the two images after B-spline registration
    moving_resampled_final = sitk.Resample(moving_resampled_affine, fixed_img, bspline_transform,
                                           INTERPOLATOR, 0.0, moving_img.GetPixelID())
    bspline_mutual_info = calculate_mutual_info(np.array(he_gray),
                                                np.array(get_pil_from_itk(moving_resampled_final)))
    verbose_print('B-spline mutual information metric: {0}'.format(bspline_mutual_info))
    performance_df.loc[idx, "Final_Mutual_Info"] = bspline_mutual_info

    # Transform the original ki67 into the aligned ki67 image  ################################################
    moving_rgb_affine = sitk_transform_rgb(ki67, he_norm, affine_transform, INTERPOLATOR)
    ki67_aligned = sitk_transform_rgb(moving_rgb_affine, he_norm, bspline_transform, INTERPOLATOR)

    # Remove backgrounds from ki67 and H&E images  ################################################
    ki67_filtered = filter_green(ki67_aligned, GREEN_FILTER_THRESHOLD)
    ki67_filtered = filter_grays(ki67_filtered, KI67_GRAY_FILTER_TOLERANCE)
    
    he_filtered = filter_green(he_norm, GREEN_FILTER_THRESHOLD)
    he_filtered = filter_grays(he_filtered, HE_GRAY_FILTER_TOLERANCE)

    # Visually compare alignment between the registered ki67 and original H&E image
    comparison_post_colour_overlay = show_alignment(he_filtered, ki67_filtered)
    verbose_save_img(comparison_post_colour_overlay.convert('RGB'),
                     os.path.join(OUTPUT_PATH, slide_name + '_comparison_post_align_colour_overlay.jpeg'), 'JPEG')
    
    
    ################################################
    positive_th_list = [240] #  < positive_th: 0,positive;  > positive_th: 1,negative
    for positive_th in positive_th_list:
        c_mask = cancer_mask(ki67_filtered, TILE_SIZE, positive_th).astype(np.bool)    
        t_mask_ki67 = tissue_mask_grabcut(ki67_filtered, TILE_SIZE, 0.5)
        t_mask_he = tissue_mask_grabcut(he_filtered, TILE_SIZE, 0.5)
        
        # Generate tissue mask with tissue common to both the KI67 and H&E image
        t_mask = np.logical_not(np.logical_not(t_mask_ki67) & np.logical_not(t_mask_he))

        # Generate uncertain mask
        u_mask = uncertain_mask(ki67_filtered, TILE_SIZE, CANCER_THRESH, NON_CANCER_THRESH)
        u_mask_filtered = np.logical_not(np.logical_not(u_mask) & np.logical_not(t_mask))

        # Filter tissue mask such that any uncertain tiles are removed
        t_mask_filtered = np.zeros(t_mask.shape)
        for x in range(t_mask.shape[0]):
            for y in range(t_mask.shape[1]):
                if t_mask[x, y] == 0 and u_mask[x, y] == 1:
                    t_mask_filtered[x, y] = False
                else:
                    t_mask_filtered[x, y] = True

        non_c_mask = np.invert(c_mask)
        non_c_mask = np.logical_not(np.logical_and(np.logical_not(non_c_mask), np.logical_not(t_mask_filtered)))

        # Merge non-cancer mask with uncertain mask
        u_mask_filtered = np.logical_not(np.logical_or(np.logical_not(non_c_mask), np.logical_not(u_mask_filtered)))
        # Blank out non cancer mask
        non_c_mask_filtered = np.ones(non_c_mask.shape, dtype=bool)
        # Cancer tile are tiles that are in the tissue and not cancer
        # Make sure all cancer tiles exist in the tissue mask
        c_mask_filtered = np.logical_not(np.logical_not(c_mask) & np.logical_not(t_mask_filtered))
        verbose_print('Cancer Slide Identified')

        overlay_ki67 = plot_mask_new(ki67_filtered, c_mask_filtered, t_mask_filtered, TILE_SIZE, False, 
                                     u_mask_filtered, width_proportion=0.05)
        verbose_save_img(overlay_ki67.convert('RGB'), os.path.join(OUTPUT_PATH, slide_name + '_KI67_overlay_%s.jpeg'% positive_th), 'JPEG')

        overlay_he = plot_mask_new(he_filtered, c_mask_filtered, t_mask_filtered, TILE_SIZE, True, 
                                   u_mask_filtered, width_proportion=0.05)
        verbose_save_img(overlay_he.convert('RGB'), os.path.join(OUTPUT_PATH, slide_name + '_HE_overlay3-%s.jpeg'% positive_th), 'JPEG')
        
        # Make Directory to save tiles
        TILES_PATH = os.path.join(OUTPUT_PATH, slide_name + '_tiles_' + str(positive_th))
        os.makedirs(TILES_PATH, exist_ok=True)
        x_tiles,y_tiles = c_mask.shape
        performance_df.loc[idx, "x_tiles_%s" % positive_th] = x_tiles
        performance_df.loc[idx, "y_tiles_%s" % positive_th] = y_tiles

        tile_coords = tile_coordinates(he_filtered, TILE_SIZE)
        
        tgen = tile_gen_at_mag_for_kfb(slide_name, tile_coords, SAVE_TILE_SIZE)
        w,h = save_tiles_for_kfb(normaliser, TILES_PATH, tgen, c_mask_filtered, t_mask_filtered, u_mask_filtered, slide_name)  
        
        non_cancer_tiles = np.invert(non_c_mask_filtered).sum()
        uncertain_tiles = np.invert(u_mask_filtered).sum()
        cancer_tiles = np.invert(c_mask_filtered).sum()

        performance_df.loc[idx, "Cancer_Tiles_%s" % positive_th] = cancer_tiles
        performance_df.loc[idx, "Uncertain_Tiles_%s" % positive_th] = uncertain_tiles
        performance_df.loc[idx, "Non_Cancer_Tiles_%s" % positive_th] = non_cancer_tiles
        performance_df.loc[idx, "Tiles_w_%s" % positive_th] = w
        performance_df.loc[idx, "Tiles_h_%s" % positive_th] = h
        
    performance_df.to_csv(os.path.join(OUTPUT_PATH, '%s_performance_metrics.csv'% slide_name))

In [ ]:
        overlay_ki67 = plot_mask_new(ki67_filtered, c_mask_filtered, t_mask_filtered, TILE_SIZE, False, 
                                     u_mask_filtered, width_proportion=0.05)
        verbose_save_img(overlay_ki67.convert('RGB'), os.path.join(OUTPUT_PATH, slide_name + '_KI67_overlay_%s.jpeg'% positive_th), 'JPEG')

        overlay_he = plot_mask_new(he_filtered, c_mask_filtered, t_mask_filtered, TILE_SIZE, True, 
                                   u_mask_filtered, width_proportion=0.05)
        verbose_save_img(overlay_he.convert('RGB'), os.path.join(OUTPUT_PATH, slide_name + '_HE_overlay3-%s.jpeg'% positive_th), 'JPEG')
        
        # Make Directory to save tiles
        TILES_PATH = os.path.join(OUTPUT_PATH, slide_name + '_tiles_' + str(positive_th))
        os.makedirs(TILES_PATH, exist_ok=True)
        x_tiles,y_tiles = c_mask.shape
        performance_df.loc[idx, "x_tiles_%s" % positive_th] = x_tiles
        performance_df.loc[idx, "y_tiles_%s" % positive_th] = y_tiles

        tile_coords = tile_coordinates(he_filtered, TILE_SIZE)
        
        tgen = tile_gen_at_mag_for_kfb(slide_name, tile_coords, SAVE_TILE_SIZE)
        w,h = save_tiles_for_kfb(normaliser, TILES_PATH, tgen, c_mask_filtered, t_mask_filtered, u_mask_filtered, slide_name)  
        
        non_cancer_tiles = np.invert(non_c_mask_filtered).sum()
        uncertain_tiles = np.invert(u_mask_filtered).sum()
        cancer_tiles = np.invert(c_mask_filtered).sum()

        performance_df.loc[idx, "Cancer_Tiles_%s" % positive_th] = cancer_tiles
        performance_df.loc[idx, "Uncertain_Tiles_%s" % positive_th] = uncertain_tiles
        performance_df.loc[idx, "Non_Cancer_Tiles_%s" % positive_th] = non_cancer_tiles
        performance_df.loc[idx, "Tiles_w_%s" % positive_th] = w
        performance_df.loc[idx, "Tiles_h_%s" % positive_th] = h